# Model Simulation

In [21]:
## Standard imports
import pandas as pd
import numpy as np
## Script imports
import simuFlares
from STL_IF import STLIF
import detectFlare
from sigma_clip import sigma_clip
## Simulation status
from IPython.display import clear_output

## Setup
# Load Data
pdcsap = pd.read_csv("../0.Data/031381302.csv", index_col = 'time').loc[:, ["pdcsap_flux"]].dropna()
# Calm interval
pdcsap = pdcsap.query("1442 <= index <= 1449")
inds = np.arange(pdcsap.shape[0])

## Flare parameters
num_flares = 5
# Base half-peak timescale: larger values => all flares last longer (relative to their amplitudes)
t_half = 4.32/120 #2.5  # e.g. 10 minutes (2-min cadence)
# Flare ampltiude (Pareto) parameters
xm = pdcsap['pdcsap_flux'].mean() * 0.005 #5                # Scale (~ x_min): Baseline amplitude (values will rarely be smaller than this)
alpha = 1.5 #1                                              # Shape: smaller => heavier tail = more large flares
offset = 0 #pdcsap['pdcsap_flux'].mean() * 0.005 #10        # Offset amplitudes (shift)
upper = pdcsap['pdcsap_flux'].mean() * 0.05 #1.0 #150       # Amplitude cap
print(upper)
# xm = 10         # Scale (~ x_min): Baseline amplitude (values will rarely be smaller than this)
# alpha = 1       # Shape: smaller => heavier tail = more large flares.
# offset = 30     # Offset amplitudes (shift)
# upper = 100     # Amplitude cap

78.22080020208084


In [22]:
## Isolation Forest parameters
contamination = 0.0025 # Expected proportion of anomalies
n_estimators = 100 # Number of trees
sample_size = "auto" # Number of samples used to train each tree
# Detrending = True
detrend = False

## Simulate
n = 100 # Number of simulations
stlif_metrics = []
sigma_metrics = []

for i in range(n):
    ## Simulation status
    clear_output(wait=True)
    print(i+1)

    ## Simulate flares
    flare_lightcurve, flare_times = simuFlares.kepler_flare(
        inds,                           # time array
        t_half,                         # base half-peak width
        num_flares,                     # number of flares
        flux_dist=simuFlares.rpareto,   # amplitude distribution
        xm=xm, alpha=alpha, offset=offset, upper=upper
    )
    # Inject flares
    data = pdcsap.copy()
    data["pdcsap_flux"] += flare_lightcurve

    ## Run model: STLIF
    data = STLIF(data, contamination=contamination, detrend=detrend, n_estimators=n_estimators, sample_size=sample_size)

    # Calculate metrics
    prec, rec, f1 = detectFlare.event_level_scores(real_flares=flare_times, y_pred=data["anomaly"].values)
    stlif_metrics.append((prec, rec, f1))

    ## Run model: STLSigmaClip
    # Note: Uses detrended series from STLIF output.
    if not detrend:
        from statsmodels.tsa.seasonal import STL
        stl = STL(pdcsap.copy(), period=240, robust=True)  # Use period=240 based on EDA
        decomposition = stl.fit()
        data = decomposition.resid.to_frame()
    anomalies = sigma_clip(data['resid'], sigma=3.0, consecutive_pts=3).ravel()

    # Calculate metrics
    prec, rec, f1 = detectFlare.event_level_scores(real_flares=flare_times, y_pred=anomalies)
    sigma_metrics.append((prec, rec, f1))

## Compute average metrics
avg_prec, avg_rec, avg_f1 = np.array(stlif_metrics).mean(axis=0)

# Print results
print(f"After {n} runs:")
print("STLIF:")
print(f"  Avg Precision: {avg_prec:.3f}")   # PPV = TP / (Detected flares = TP + FP) - How many predictions were accurate
print(f"  Avg Recall:    {avg_rec:.3f}")    # Sensitivity = TP / (True flares = TP + FN) - How many of the true flares did you actually get
print(f"  Avg F1 Score:  {avg_f1:.3f}")     # PPV x Sensitivity

## Compute average metrics
avg_prec, avg_rec, avg_f1 = np.array(sigma_metrics).mean(axis=0)

print("3-3sigma:")
print(f"  Avg Precision: {avg_prec:.3f}")
print(f"  Avg Recall:    {avg_rec:.3f}")
print(f"  Avg F1 Score:  {avg_f1:.3f}")

100
After 100 runs:
STLIF:
  Avg Precision: 0.175
  Avg Recall:    0.418
  Avg F1 Score:  0.246
3-3sigma:
  Avg Precision: 0.000
  Avg Recall:    0.000
  Avg F1 Score:  0.000


## Hyperparameter Tuning

In [8]:
## Isolation Forest parameters
# Expected proportion of anomalies
contamination_values = [0.001, 0.002, 0.0035, 0.005]
# Number of trees
n_estimators_values = [100, 200, 300]
# Number of samples used to train each tree
max_samples_values = ["auto"]

## Simulate
n_runs = 25 # Number of simulations
results = []
# Counter
k = 1
import itertools
total_k = len(list(itertools.product(contamination_values, n_estimators_values, max_samples_values))) # Total parameter combinations

# Create a small param grid
param_grid = []
for c in contamination_values:
    for ne in n_estimators_values:
        for ms in max_samples_values:
            param_grid.append((c, ne, ms))

for (contamination, n_est, m_samp) in param_grid:
    ## Simulation status
    clear_output(wait=True)
    print("Combination: ", k, "/", total_k, " (contamination=", contamination, ", n_est=", n_est, ", m_samp=", m_samp, ")", sep="")
    k += 1

    ## Setup
    run_metrics = []
    
    for run_i in range(n_runs):
        ## Simulate flares
        flare_lightcurve, flare_times = simuFlares.kepler_flare(
            inds,                           # time array
            t_half,                         # base half-peak width
            num_flares,                     # number of flares
            flux_dist=simuFlares.rpareto,   # amplitude distribution
            xm=xm, alpha=alpha, offset=offset, upper=upper
        )
        # Inject flares
        data = pdcsap.copy()
        data["pdcsap_flux"] += flare_lightcurve

        ## Run model: STLIF
        data = STLIF(data, contamination=contamination, n_estimators=n_est, sample_size=m_samp)
        
        # Calculate metrics
        prec, rec, f1 = detectFlare.event_level_scores(real_flares=flare_times, y_pred=data["anomaly"].values)
        run_metrics.append((prec, rec, f1))
    
    # Average performance over n_runs
    avg_prf = np.mean(run_metrics, axis=0)
    result_dict = {
        "contamination": contamination,
        "n_estimators": n_est,
        "max_samples": m_samp,
        "avg_precision": avg_prf[0],
        "avg_recall":    avg_prf[1],
        "avg_f1_score":  avg_prf[2],
    }
    results.append(result_dict)

# Sort results by F1
results.sort(key=lambda x: x["avg_f1_score"], reverse=True)

Combination: 12/12 (contamination=0.005, n_est=300, m_samp=auto)


In [11]:
# Print top results
print("Top 5 Hyperparam Combos (by F1):")
for row in results[:5]:
    print(row)

Top 5 Hyperparam Combos (by F1):
{'contamination': 0.001, 'n_estimators': 300, 'max_samples': 'auto', 'avg_precision': 0.44066666666666665, 'avg_recall': 0.39199999999999996, 'avg_f1_score': 0.41288888888888875}
{'contamination': 0.001, 'n_estimators': 100, 'max_samples': 'auto', 'avg_precision': 0.392, 'avg_recall': 0.32000000000000006, 'avg_f1_score': 0.3486666666666667}
{'contamination': 0.002, 'n_estimators': 100, 'max_samples': 'auto', 'avg_precision': 0.2692380952380952, 'avg_recall': 0.45600000000000007, 'avg_f1_score': 0.3363090243090243}
{'contamination': 0.001, 'n_estimators': 200, 'max_samples': 'auto', 'avg_precision': 0.3380000000000002, 'avg_recall': 0.3200000000000001, 'avg_f1_score': 0.3280000000000001}
{'contamination': 0.002, 'n_estimators': 200, 'max_samples': 'auto', 'avg_precision': 0.22125396825396826, 'avg_recall': 0.4000000000000001, 'avg_f1_score': 0.28414652014652014}
